In [2]:
import sys
sys.path.append('/usr/local/python')
from  openpose  import pyopenpose as op
import os
import json
import glob
import subprocess
from shlex import quote, split
import numpy as np

In [4]:
video_paths = glob.glob('/mnt/arnav/oops_dataset/oops_video/train/*.mp4') + glob.glob('/mnt/arnav/oops_dataset/oops_video/val/*.mp4')
openpose_path = '/mnt1/arnav/pose-estimation/openpose/'
video_dir = '/mnt1/arnav/pose-estimation/oops-videos/'
proj_dir = '/mnt1/arnav/pose-estimation/'
examples = video_paths[-1]

In [36]:
videonames = set(np.load('/home/exx/arnav/transmodel-pytorch/Oops-Annotations/videoname.npy', allow_pickle=True))

## Removing spaces and renaming files

In [37]:
copy_paths = []
for path in video_paths:
    base_path = os.path.splitext(os.path.basename(path))[0]
    if base_path in videonames:
        copy_paths.append(path)

In [41]:
mapper = {}

cmd = '-exec cp {} /mnt1/arnav/pose-estimation/oops-videos/ ;'
for i,files in enumerate(copy_paths):
    os.system(f'cp {quote(files)} {os.path.join(video_dir,"video"+str(i)+".mp4")}')
    mapper[f'video{i}'] = os.path.splitext(os.path.basename(files))[0]

## Testing open pose library

In [ ]:
cd openpose && ./build/examples/openpose/openpose.bin --video ../video.mp4 --write_json ./output/ --display 0  --write_video ../openpose.avi

In [53]:
example = glob.glob(f'{video_dir}*.mp4')[0]
base_name = os.path.splitext(os.path.basename(example))[0]
cmd = f'cd {openpose_path} && ./build/examples/openpose/openpose.bin --video {example} --write_json ../results/{base_name} --display 0  --write_video ../results/{base_name}/processed.avi'

## Processing data in Deepsort format

In [11]:
import cv2
import shutil

In [13]:
results_dir = glob.glob(os.path.join(proj_dir,'results3/*'))
for directory in results_dir:
    if os.path.exists(os.path.join(directory,'numpy')):
        shutil.rmtree(os.path.join(directory,'numpy'))
    os.mkdir(os.path.join(directory,'numpy'))
    keypoints_json_files = glob.glob(os.path.join(directory,'*.json'))
    # iterating over kepoints for each frame
    for i,json_file in enumerate(sorted(keypoints_json_files)):
        index = int(os.path.splitext(os.path.basename(json_file))[0].split('_')[1])
        with open(json_file,'r') as f:
            person_kp_list = []
            keypoints = json.load(f)
            # number of detected people
            num_people = len(keypoints['people'])
            for ix in range(num_people):
                person_kp_list.append(keypoints['people'][ix]['pose_keypoints_2d'])
            person_kp_list = np.array(person_kp_list).reshape(num_people, 18, 3)
            with open(os.path.join(directory,'numpy',f'frame_{index}.npy'),'wb') as f:
                np.save(f, person_kp_list)


In [32]:
example = glob.glob(os.path.join(glob.glob(os.path.join(proj_dir,'results/*'))[0],'numpy/*'))

In [41]:
int(os.path.splitext(os.path.basename(example[0]))[0].split('_')[1])

('frame_417', '.npy')

## Converting avi to mp4 videos

In [10]:
results_dir = glob.glob(os.path.join(proj_dir,'results3/*'))
for directory in results_dir:
    if not os.path.exists(os.path.join(directory,'processed.mp4')):
        print('no')
    # !cd {directory} && ffmpeg -y -loglevel info -i processed.avi processed.mp4

'/mnt1/arnav/pose-estimation/results3/video390'